In [ ]:
import random
import math
import numpy as np
import pandas as pd
from datetime import timedelta
from copy import deepcopy
from openpyxl import Workbook
from openpyxl.styles import Border, Side, Alignment

# Основные классы

In [ ]:
class BusDriver:
    def __init__(self, name, shift_type, bus, home_depot):
        self.name = name # Номер водителя
        self.shift_type = shift_type  # Тип смены
        self.bus = bus  # Привязанный автобус
        self.home_depot = home_depot  # Домашнее депо
        self.on_lunch = False  # Перерыв
        self.working_time = timedelta(hours=0)  # Время работы за день
        self.days_worked = 0  # Дни, отработанные подряд
        self.global_resting_time = timedelta(hours=0)  # Время отдыха за весь день
        self.resting_time = timedelta(minutes=0)  # Время, которое водитель на отдыхе
        self.between_shifts_time = timedelta(hours=11) # Количество времени между сменами

        '''У 12часового водителя 2 перерыва'''
        if self.shift_type == timedelta(hours=12):
            self.daily_breaks = 2  # Количество перерывов за смену
            self.break_duration = timedelta(minutes=30)
            # Поскольку 12 часовой водитель работает 2 через 2 добавляем флаг, показывающий может ли работать водитель
            self.can_work_today = True
            self.day_off = timedelta(hours=0) # Отсчитываем выходной
        else:
            self.daily_breaks = 1 # Выходной 8 часового водителя
            self.break_duration = timedelta(minutes=60)

    def assign_bus_to_driver(self, bus_pool):
        """
        Назначает автобус водителю из пула свободных автобусов.
        """
        if bus_pool:
            self.bus = bus_pool.pop(0)
            return True
        return False

    def release_bus_from_driver(self, bus_pool):
        """
        Освобождает автобус от водителя и возвращает его в пул.
        """
        if self.bus:
            bus_pool.append(self.bus)
            self.bus = None


    def take_break(self, bus_pool, drivers_on_break):
        '''Если вызывается функция take_break водитель отправляется на перерыв'''
        self.release_bus_from_driver(bus_pool)
        self.daily_breaks -= 1
        self.on_lunch = True
        drivers_on_break.append(self)
        # print(f"Водитель {self.name} отправляется на перерыв. Осталось перерывов: {self.daily_breaks}")


    def end_break(self, drivers_on_break, bus_pool, active_drivers, ended_drivers):
        '''Функция end_break сравнивает время отдыха водителя с временем перерыва
        и отправляет его на работу'''
        if self.on_lunch:
            self.resting_time += timedelta(minutes=1)
            if self.resting_time >= self.break_duration and bus_pool:
                drivers_on_break.remove(self) # Удаляем водителя из списка отдыхающих водителей
                self.assign_bus_to_driver(bus_pool)
                self.on_lunch = False
                self.global_resting_time = self.resting_time
                self.resting_time = timedelta(minutes=0)
            if self.resting_time > timedelta(hours=1):
                self.on_lunch = False
                self.global_resting_time = self.resting_time
                self.resting_time = timedelta(minutes=0)
                drivers_on_break.remove(self)
                self.end_of_the_day(active_drivers, ended_drivers, bus_pool)


    def drive_bus(self, time_to_next, num_stations, dispatch_interval):
        '''Вызываем функцию move привязанного к водителю автобуса'''
        if self.bus.move(num_stations, dispatch_interval):  # Если автобус достиг остановки
            return True  # Автобус остановился
        return False  # Автобус в пути

    def update_work_status(self):
        '''Проверяет сколько дней подряд отработал водитель'''
        if self.shift_type == timedelta(hours=12):
            self.days_worked += 1
            if self.days_worked == 2:  # Если отработал 2 дня подряд
                self.can_work_today = False
                self.day_off = timedelta(hours=48)
                self.days_worked = 0  # Сбрасываем счётчик после 2 выходных


    def update_day_off(self):
        '''Обновляет сколько времени на выходном проводит 12 часовой водитель'''
        if self.shift_type == timedelta(hours=12):
            if not self.can_work_today:
                self.day_off -= timedelta(minutes=1)
                if self.day_off <= timedelta(minutes=0):
                    self.can_work_today = True
                    self.day_off = timedelta(hours=48)


    def end_of_the_day(self, active_drivers, ended_drivers, bus_pool):
        '''Обновляет поля водителя после завершения смены'''
        self.working_time = timedelta(hours=0)
        self.on_lunch = False
        self.global_resting_time = timedelta(hours=0)
        if self.shift_type == timedelta(hours=12):
            self.daily_breaks = 2  # Количество перерывов за смену
            self.update_work_status()
        else:
            self.daily_breaks = 1
        if self.bus:
            self.release_bus_from_driver(bus_pool)
        ended_drivers.append(self)
        active_drivers.remove(self)



    def is_allowed_to_work(self, time_road, current_time):
        if self.bus:
            if self.bus.current_station == self.home_depot:
                current_hour = (((current_time.days) * 24) + ((current_time.seconds + time_road.seconds) // 3600)) % 24
                if self.shift_type == timedelta(hours=8):
                    return (self.working_time + time_road) < self.shift_type and (5 <= current_hour <= 22)
                return (self.working_time + time_road) < self.shift_type - timedelta(hours=1)
            else:
                return True
        else:
            return True

In [ ]:
class Bus:
    def __init__(self, number, current_station, direct):
        self.number = number # Номер автобуса
        self.current_station = current_station  # Текущая остановка
        self.direct = direct # Флаг, который показывает в какую сторону едет автобус
        self.onStation = False # Флаг, который показывает, что мы находимся на станции
        self.time_to_next = timedelta(minutes=5) # Параметр, который указывает сколько ехать до следующей остановки

    def move(self, num_stations, dispatch_interval):
        '''Функция, которая отвечает за движение автобуса к следующей станции'''
        self.time_to_next -= timedelta(minutes=1) # Проходит минута и мы ближе к следующей остановке
        self.onStation = False # Флаг, который показывает, что мы на остановке

        '''Если time_to_next = 0, то мы доехали до следующей станции
        В таком случае, если мы не в депо, то обновляем нашу станцию.
        Если в депо - разворачиваемся и едем в обратную сторону'''
        if self.time_to_next == timedelta(minutes=0):
            if self.current_station != 0 and self.current_station != num_stations + 1:
                self.onStation = True

            if self.direct:
                self.current_station += 1
            else:
                self.current_station -= 1

            self.time_to_next = timedelta(minutes=5)

            if (self.current_station == 0 and self.direct == False) or (self.current_station == num_stations + 1 and self.direct == True):
                self.direct = not self.direct
                self.time_to_next = timedelta(minutes=5)


            return True  # Автобус достиг остановки
        return False  # Автобус продолжает движение

In [ ]:
class BusStation:
    def __init__(self, station_id, direct):
        self.station_id = station_id  # Уникальный идентификатор остановки
        self.direct = direct  # Направление движения (True - прямое, False - обратное)

# Параметры симуляции

In [ ]:
n_of_stations = 11

n_of_bus = 4
n_of_reverse_bus = 4

n_of_drivers_eight_shift = 15
n_of_reverse_drivers_eight_shift = 15

n_of_drivers_twelve_shift = 15
n_of_reverse_drivers_twelve_shift = 15

# Метод инициализации

In [ ]:
def initialize(n_of_stations, n_of_bus, n_of_reverse_bus, n_of_drivers_eight_shift, n_of_reverse_drivers_eight_shift,
               n_of_drivers_twelve_shift, n_of_reverse_drivers_twelve_shift):
    stations = []
    numbers = [i for i in range(1, 200)]
    for i in range(1, n_of_stations + 1):
        stations.append(BusStation(i, True))
        stations.append(BusStation(i, False))

    direct_buses = [Bus(i, 0, True) for i in range(n_of_bus)]
    reverse_buses = [Bus(i + n_of_bus, n_of_stations + 1, False) for i in range(n_of_reverse_bus)]

    drivers = []
    reverse_drivers = []
    # Создаем 8-часовых водителей для прямого направления
    total_drivers = n_of_drivers_eight_shift + n_of_reverse_drivers_eight_shift + n_of_drivers_twelve_shift + n_of_reverse_drivers_twelve_shift
    for i in range(n_of_drivers_eight_shift):
        drivers.append(BusDriver(name=numbers.pop(0), shift_type=timedelta(hours=8), bus=None, home_depot=0))

    # Создаем 8-часовых водителей для обратного направления
    for i in range(n_of_reverse_drivers_eight_shift):
        bus = reverse_buses[i % len(reverse_buses)]
        reverse_drivers.append(BusDriver(name=numbers.pop(0), shift_type=timedelta(hours=8), bus=None, home_depot=n_of_stations + 1))

    # Создаем 12-часовых водителей для прямого направления
    for i in range(n_of_drivers_twelve_shift):
        drivers.append(BusDriver(name=numbers.pop(0), shift_type=timedelta(hours=12), bus=None, home_depot=0))

    # Создаем 12-часовых водителей для обратного направления
    for i in range(n_of_reverse_drivers_twelve_shift):
        reverse_drivers.append(BusDriver(name=numbers.pop(0), shift_type=timedelta(hours=12), bus=None, home_depot=n_of_stations + 1))

    return stations, direct_buses, reverse_buses, drivers, reverse_drivers


# Вспомогательные методы для симуляции

In [ ]:
def get_dispatch_interval(current_time, total_buses, road_time, peak_multiplier=1, regular_multiplier=2, night_multiplier=4):
    # Определяем текущий час
    total_minutes = current_time.total_seconds() // 60
    current_hour = int(total_minutes // 60) % 24

    # Рассчитываем минимально возможный интервал
    if total_buses > 0:
        base_interval = road_time / total_buses  # Интервал для равномерного покрытия маршрута
    else:
        return timedelta(minutes=road_time)  # Если автобусов нет, интервал равен времени маршрута


    current_day = current_time.days % 7
    if 0 <= current_day <= 4: # В будни
    # Применяем множитель в зависимости от времени суток
        if 6 <= current_hour < 9 or 17 <= current_hour < 20:  # Часы пик
            adjusted_interval = base_interval * peak_multiplier
        elif 9 <= current_hour < 17:  # Обычное время
            adjusted_interval = base_interval * regular_multiplier
        else:  # Ночное время
            adjusted_interval = base_interval * night_multiplier

        return timedelta(minutes=adjusted_interval)

    else: # В выходные
        if 10 <= current_hour < 23:
            adjusted_interval = base_interval * regular_multiplier
        else:
            adjusted_interval = base_interval * night_multiplier

        return timedelta(minutes=adjusted_interval)


In [ ]:
def handle_drivers_actions(active_drivers,
                           ended_drivers,
                           drivers_on_break,
                           stations,
                           direct_buses,
                           reverse_buses,
                           n_of_stations,
                           df,
                           current_time):
    time_str = f"{str(current_time.days % 7)}, {str(current_time - timedelta(hours=current_time.days * 24))}"

    if time_str not in df.index:
        df.loc[time_str] = [pd.NA]*len(df.columns)

    dispatch_interval = get_dispatch_interval(current_time, 4, (n_of_stations + 1) * 5)

    for driver in active_drivers[:]:
        if driver.name not in df.columns:
            df[driver.name] = pd.NA
            # Заполним текущее время для нового водителя
            # Начал работу в депо
            df.at[time_str, driver.name] = [
            f"Начал работу в депо {driver.home_depot}",
            f"Смена: {driver.shift_type}",
            f"Автобус: {driver.bus.number}"
        ]
        # Проверка, может ли водитель продолжать работать
        if not driver.is_allowed_to_work(timedelta(minutes=5) * 2 * n_of_stations, current_time):

            # Водитель завершил смену
            if driver.bus.direct:
                bus_pool = direct_buses
            else:
                bus_pool = reverse_buses
            df.at[time_str, driver.name] = [
            f"Закончил работу в депо {driver.home_depot}",
            f"Смена: {driver.shift_type}",
            f"Автобус: {driver.bus.number}"
        ]
            driver.end_of_the_day(active_drivers, ended_drivers, bus_pool)
            continue

        if driver.on_lunch:
            # Водитель на перерыве, проверяем, завершил ли перерыв
            df.at[time_str, driver.name] = [
            "На перерыве",
            f"Смена: {driver.shift_type}",
            f"Автобус: {'Не назначен'}"
        ]
            driver.end_break(drivers_on_break, (direct_buses if not driver.home_depot else reverse_buses), active_drivers, ended_drivers)
            # Если перерыв завершён, driver.on_lunch станет False внутри end_break
            continue
        else:
            if driver.shift_type == timedelta(hours=8):
                if driver.working_time >= timedelta(hours=3) and driver.daily_breaks > 0 and driver.bus.current_station == driver.home_depot:
                    driver.take_break((direct_buses if driver.bus.direct else reverse_buses), drivers_on_break)
                    continue
            else:
                if (
                    driver.working_time >= timedelta(hours=3)
                    and driver.daily_breaks == 2
                    and driver.bus.current_station == driver.home_depot
                ) or (
                    driver.working_time >= timedelta(hours=7)
                    and driver.daily_breaks == 1
                    and driver.bus.current_station == driver.home_depot
                ):
                    driver.take_break((direct_buses if driver.bus.direct else reverse_buses), drivers_on_break)
                    continue

        # Водитель не на перерыве и не завершил смену, увеличиваем время работы
        driver.working_time += timedelta(minutes=1)

        # Попытка проехать к следующей остановке
        previous_station_id = driver.bus.current_station
        reached_station = driver.drive_bus(driver.bus.time_to_next, n_of_stations, dispatch_interval)

        if reached_station:
            station_id = driver.bus.current_station
            # Если достигнута остановка (не депо), загружаем/выгружаем пассажиров
            if station_id  in (0, n_of_stations + 1):
                df.at[time_str, driver.name] = [
                f"В депо {station_id}",
                f"Смена: {driver.shift_type}",
                f"Автобус: {driver.bus.number}"
            ]
        else:
            df.at[time_str, driver.name] = [
            f"Едет до депо {(n_of_stations + 1) if driver.bus.direct else 0}",
            f"Смена: {driver.shift_type}",
            f"Автобус: {driver.bus.number}"
        ]
    return df


In [ ]:
def can_dispatch_8_hour_driver(current_hour, current_day):
        """
        Проверяет, можно ли выпустить 8-часового водителя.
        """
        return current_day < 5 and 5 <= current_hour < 22

In [ ]:
def get_driver(direction, ended_drivers, drivers_pool, current_hour, current_day):

        allow_8_hour = (5 <= current_hour < 22 and current_day in range(0, 5))
        for ed in ended_drivers:
            is_direct = (ed.home_depot == 0)
            if allow_8_hour and ed.shift_type == timedelta(hours=8):
                if (is_direct == direction and ed.between_shifts_time <= timedelta(minutes=0)):
                    # Сбрасываем параметры водителя
                    ed.between_shifts_time = timedelta(hours=11)
                    ended_drivers.remove(ed)
                    return ed
            elif not allow_8_hour and ed.shift_type == timedelta(hours=12):
                if (is_direct == direction and ed.can_work_today and ed.between_shifts_time <= timedelta(minutes=0)):
                    ed.between_shifts_time = timedelta(hours=11)

                    ended_drivers.remove(ed)
                    return ed

        # 2. Пытаемся найти водителя с нужной сменой в drivers_pool
        for drv in drivers_pool:
            if allow_8_hour and drv.shift_type == timedelta(hours=8):
                drivers_pool.remove(drv)
                return drv
            elif not allow_8_hour and drv.shift_type == timedelta(hours=12):
                drivers_pool.remove(drv)
                return drv
        # Если подходящих водителей нет
        return None


In [ ]:
def check_and_dispatch_new_drivers(current_time,
                                   ended_drivers,
                                   active_drivers,
                                   drivers_on_break,
                                   drivers,
                                   reverse_drivers,
                                   direct_buses,
                                   reverse_buses,
                                   last_dispatch_time_direct,
                                   last_dispatch_time_reverse,
                                   n_of_stations):

    total_minutes = current_time.total_seconds() // 60
    current_hour = int(total_minutes // 60) % 24
    dispatch_interval = get_dispatch_interval(current_time, 4, (n_of_stations + 1) * 5)

    interval_minutes = dispatch_interval.total_seconds() // 60
    required_buses = math.ceil((n_of_stations * 5) / interval_minutes)


    current_day = current_time.days % 7
    allow_8_hour = can_dispatch_8_hour_driver(current_hour, current_day)



    needed_buses = required_buses - len(active_drivers) // 2

    for _ in range(needed_buses):
        if last_dispatch_time_direct + dispatch_interval <= current_time or last_dispatch_time_direct == timedelta(hours=0):
            last_dispatch_time_direct = current_time
            drv = get_driver(True, ended_drivers, drivers, current_hour, current_day)
            if drv:
                if drv.assign_bus_to_driver(direct_buses):
                    active_drivers.append(drv)

            # Обратное направление
            drv_rev = get_driver(False, ended_drivers, reverse_drivers, current_hour, current_day)
            if drv_rev:
                if drv_rev.assign_bus_to_driver(reverse_buses):
                    active_drivers.append(drv_rev)




    return last_dispatch_time_direct, last_dispatch_time_reverse


# Основной метод симуляции

In [ ]:
def simulate_time(simulation_duration,
                  n_of_stations,
                  n_of_buses,
                  n_of_reverse_buses,
                  n_of_drivers_eight_shift,
                  n_of_reverse_drivers_eight_shift,
                  n_of_drivers_twelve_shift,
                  n_of_reverse_drivers_twelve_shift):

    stations, direct_buses, reverse_buses, drivers, reverse_drivers = initialize(
        n_of_stations, n_of_buses, n_of_reverse_buses,
        n_of_drivers_eight_shift, n_of_reverse_drivers_eight_shift,
        n_of_drivers_twelve_shift, n_of_reverse_drivers_twelve_shift
    )

    active_drivers = []
    ended_drivers = []
    drivers_on_break = []
    df = pd.DataFrame(dtype=object)
    df = pd.DataFrame(columns=["placeholder"])
    current_time = timedelta(hours=0, minutes=30)
    simulation_end = timedelta(minutes=simulation_duration)

    time_str = str(current_time)


    last_dispatch_time_direct = timedelta(seconds=0)
    last_dispatch_time_reverse = timedelta(seconds=0)
    simulation_end += current_time

    while current_time < simulation_end:

        current_hour = (current_time.total_seconds() // 3600) % 24


            # Обновление состояний водителей и автобусов
        df = handle_drivers_actions(active_drivers,
                                    ended_drivers,
                                    drivers_on_break,
                                    stations,
                                    direct_buses,
                                    reverse_buses,
                                    n_of_stations,
                                    df,
                                    current_time)

        if ended_drivers:
            for driver in ended_drivers:
                driver.between_shifts_time -= timedelta(minutes=1)
                driver.update_day_off()

        last_dispatch_time_direct, last_dispatch_time_reverse = check_and_dispatch_new_drivers(current_time,
                                   ended_drivers,
                                   active_drivers,
                                   drivers_on_break,
                                   drivers,
                                   reverse_drivers,
                                   direct_buses,
                                   reverse_buses,
                                   last_dispatch_time_direct,
                                   last_dispatch_time_reverse,
                                   n_of_stations)

        # Увеличиваем текущее время
        current_time += timedelta(minutes=1)





    print("Симуляция завершена.")

    print("Всего водителей:", len(active_drivers) + len(ended_drivers))
    return df



# Параметры симуляции
df = simulate_time(
    simulation_duration=10000,
    n_of_stations=n_of_stations,
    n_of_buses=n_of_bus,
    n_of_reverse_buses=n_of_reverse_bus,
    n_of_drivers_eight_shift=n_of_drivers_eight_shift,
    n_of_reverse_drivers_eight_shift=n_of_reverse_drivers_eight_shift,
    n_of_drivers_twelve_shift=n_of_drivers_twelve_shift,
    n_of_reverse_drivers_twelve_shift=n_of_reverse_drivers_twelve_shift,
)

Симуляция завершена.
Всего водителей: 24


In [ ]:
df = df.reset_index(names="Time_index")

In [ ]:
df

,Time_index,placeholder,31,46,1,16,2,17,3,18,...,7,22,32,47,33,48,34,49,35,50
0,"0, 0:30:00",NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,"0, 0:31:00",NaN,"[Едет до депо 12, Смена: 12:00:00, Автобус: 0]","[Едет до депо 0, Смена: 12:00:00, Автобус: 4]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,"0, 0:32:00",NaN,"[Едет до депо 12, Смена: 12:00:00, Автобус: 0]","[Едет до депо 0, Смена: 12:00:00, Автобус: 4]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,"0, 0:33:00",NaN,"[Едет до депо 12, Смена: 12:00:00, Автобус: 0]","[Едет до депо 0, Смена: 12:00:00, Автобус: 4]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,"0, 0:34:00",NaN,"[Едет до депо 12, Смена: 12:00:00, Автобус: 0]","[Едет до депо 0, Смена: 12:00:00, Автобус: 4]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,"6, 23:05:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[Едет до депо 0, Смена: 12:00:00, Автобус: 3]","[Едет до депо 12, Смена: 12:00:00, Автобус: 7]","[Едет до депо 12, Смена: 12:00:00, Автобус: 1]","[Едет до депо 0, Смена: 12:00:00, Автобус: 5]"
9996,"6, 23:06:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Едет до депо 12, Смена: 12:00:00, Автобус: 1]","[Едет до депо 0, Смена: 12:00:00, Автобус: 5]"
9997,"6, 23:07:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[Едет до депо 0, Смена: 12:00:00, Автобус: 3]","[Едет до депо 12, Смена: 12:00:00, Автобус: 7]","[Едет до депо 12, Смена: 12:00:00, Автобус: 1]","[Едет до депо 0, Смена: 12:00:00, Автобус: 5]"
9998,"6, 23:08:00",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,"[Едет до депо 0, Смена: 12:00:00, Автобус: 3]","[Едет до депо 12, Смена: 12:00:00, Автобус: 7]",NaN,NaN


In [ ]:
def auto_adjust_column_width(sheet):
    for col in sheet.iter_cols(min_row=1, max_row=sheet.max_row, min_col=1, max_col=sheet.max_column):
        max_length = 0
        col_letter = col[0].coordinate.split(":")[0][0]  # Определяем букву столбца через координаты
        for cell in col:
            try:
                if cell.value:  # Проверяем, есть ли значение в ячейке
                    max_length = max(max_length, len(str(cell.value)))
            except:
                pass
        adjusted_width = max_length + 2  # Добавляем немного пространства
        sheet.column_dimensions[col_letter].width = adjusted_width




def add_borders(sheet):
    """Добавляет только внешние толстые границы для групп ячеек."""
    thick_border = Border(
        left=Side(style='thick'),
        right=Side(style='thick'),
        top=Side(style='thick'),
        bottom=Side(style='thick')
    )

    # Границы вокруг группы A1:B3 (Водитель и Вид смены)
    for row in range(1, 4):  # Ряды 1-3
        for col in range(1, 3):  # Колонки A и B
            cell = sheet.cell(row=row, column=col)
            if row == 1:  # Верхняя граница
                cell.border = Border(top=thick_border.top, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
            if row == 3:  # Нижняя граница
                cell.border = Border(bottom=thick_border.bottom, left=cell.border.left, right=cell.border.right, top=cell.border.top)
            if col == 1:  # Левая граница
                cell.border = Border(left=thick_border.left, top=cell.border.top, right=cell.border.right, bottom=cell.border.bottom)
            if col == 2:  # Правая граница
                cell.border = Border(right=thick_border.right, top=cell.border.top, left=cell.border.left, bottom=cell.border.bottom)

    # Границы для дней недели (C1-F2, G1-J2 и т.д.)
    days_start_col = 3
    days_per_group = 4  # 4 колонки на день: Автобус, Начало, Конец, Действие

    for i in range(7):  # 7 дней недели
        start_col = days_start_col + i * days_per_group
        end_col = start_col + days_per_group - 1

        # Границы для заголовков дней недели
        for row in range(1, 3):  # Ряды 1-2
            for col in range(start_col, end_col + 1):
                cell = sheet.cell(row=row, column=col)
                if row == 1:  # Верхняя граница
                    cell.border = Border(top=thick_border.top, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
                if row == 2:  # Нижняя граница
                    cell.border = Border(bottom=thick_border.bottom, left=cell.border.left, right=cell.border.right, top=cell.border.top)
                if col == start_col:  # Левая граница
                    cell.border = Border(left=thick_border.left, top=cell.border.top, right=cell.border.right, bottom=cell.border.bottom)
                if col == end_col:  # Правая граница
                    cell.border = Border(right=thick_border.right, top=cell.border.top, left=cell.border.left, bottom=cell.border.bottom)

        # Границы вокруг данных для каждого дня недели
        for row in range(3, sheet.max_row + 1):  # Начиная с 3 строки
            for col in range(start_col, end_col + 1):
                cell = sheet.cell(row=row, column=col)
                if row == 3:  # Верхняя граница данных
                    cell.border = Border(top=thick_border.top, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
                if row == sheet.max_row:  # Нижняя граница данных
                    cell.border = Border(bottom=thick_border.bottom, left=cell.border.left, right=cell.border.right, top=cell.border.top)
                if col == start_col:  # Левая граница
                    cell.border = Border(left=thick_border.left, top=cell.border.top, right=cell.border.right, bottom=cell.border.bottom)
                if col == end_col:  # Правая граница
                    cell.border = Border(right=thick_border.right, top=cell.border.top, left=cell.border.left, bottom=cell.border.bottom)

        # Границы между группами столбцов
        for row in range(1, sheet.max_row + 1):
            left_cell = sheet.cell(row=row, column=start_col)
            left_cell.border = Border(left=thick_border.left, top=left_cell.border.top, bottom=left_cell.border.bottom)



def add_summary_sheet(workbook, job_result_df, driver_columns):
    """Добавляет лист 'Итоги' с агрегированной информацией."""
    # Создаем лист "Итоги" в начале
    summary_sheet = workbook.create_sheet(title="Итоги", index=0)

    # Общее количество водителей
    total_drivers = len(driver_columns)

    # Фильтрация по сменам
    def contains_shift(data, shift_time):
        """Проверяет, содержит ли список или строка время смены."""
        if isinstance(data, list):
            return any(f"Смена: {shift_time}" in str(item) for item in data)
        return False

    drivers_8_hour_shift = sum(
        job_result_df[col].apply(lambda x: contains_shift(x, "8:00:00")).any()
        for col in driver_columns
    )
    drivers_12_hour_shift = sum(
        job_result_df[col].apply(lambda x: contains_shift(x, "12:00:00")).any()
        for col in driver_columns
    )

    # Количество водителей и автобусов по дням недели
    days_of_week = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]
    drivers_per_day = {day: 0 for day in days_of_week}
    buses_per_day = {day: set() for day in days_of_week}  # Для подсчета уникальных автобусов
    all_buses = set()  # Для общего количества уникальных автобусов

    for col in driver_columns:
        for day_idx, day_name in enumerate(days_of_week):
            day_data = job_result_df[
                (job_result_df['Day'] == day_idx) & job_result_df[col].notna()
            ][col]
            drivers_per_day[day_name] += day_data.apply(
                lambda x: isinstance(x, list) and any("Смена:" in str(item) for item in x)
            ).any()
            buses_for_day = {
                item.split(": ")[-1] for sublist in day_data.dropna()
                for item in sublist if "Автобус:" in item
            }
            buses_per_day[day_name].update(buses_for_day)
            all_buses.update(buses_for_day)

    # Записываем данные на лист "Итоги"
    summary_sheet.append(["Общие данные"])
    summary_sheet.append(["Общее количество водителей", total_drivers])
    summary_sheet.append(["Общее количество водителей с 8-часовой сменой", drivers_8_hour_shift])
    summary_sheet.append(["Общее количество водителей с 12-часовой сменой", drivers_12_hour_shift])
    # summary_sheet.append(["Общее количество уникальных автобусов", len(all_buses)])
    summary_sheet.append([])

    # Записываем данные по дням недели
    summary_sheet.append(["Количество водителей по дням недели"])
    summary_sheet.append(["День недели", "Количество водителей"])
    for day, count in drivers_per_day.items():
        summary_sheet.append([day, count])

    # Автоширина столбцов
    auto_adjust_column_width(summary_sheet)





def excel_schedule(job_result_df, output_file):
    # Разделяем день и время
    job_result_df[['Day', 'Time']] = job_result_df['Time_index'].str.split(', ', expand=True)
    job_result_df['Day'] = job_result_df['Day'].astype(int)  # Преобразуем день в целое число
    job_result_df['Time'] = job_result_df['Time'].str.strip()  # Очищаем строки времени

    # Ищем столбцы водителей
    driver_columns = [col for col in job_result_df.columns if col not in ['Time_index', 'placeholder', 'Day', 'Time']]

    # Дни недели
    days_of_week = ["Понедельник", "Вторник", "Среда", "Четверг", "Пятница", "Суббота", "Воскресенье"]

    # Создаем Excel файл
    workbook = Workbook()
    add_summary_sheet(workbook, job_result_df, driver_columns)

    for idx, driver_name in enumerate(driver_columns, start=1):
        # Создаем отдельный лист для каждого водителя
        sheet = workbook.create_sheet(title=str(idx))
        sheet.title = f"Driver_{idx}"

        # Заполняем заголовки
        sheet.merge_cells(start_row=1, start_column=1, end_row=2, end_column=1)
        sheet["A1"] = "Водитель"
        sheet.merge_cells(start_row=1, start_column=2, end_row=2, end_column=2)
        sheet["B1"] = "Вид смены"

        col_idx = 3  # Начало столбцов для дней недели
        for day in days_of_week:
            sheet.merge_cells(start_row=1, start_column=col_idx, end_row=1, end_column=col_idx + 3)
            sheet.cell(row=1, column=col_idx).value = day
            sheet.cell(row=2, column=col_idx).value = "Автобус"
            sheet.cell(row=2, column=col_idx + 1).value = "Начало"
            sheet.cell(row=2, column=col_idx + 2).value = "Конец"
            sheet.cell(row=2, column=col_idx + 3).value = "Действие"
            col_idx += 4

        # Центрируем заголовки
        for row in sheet.iter_rows(min_row=1, max_row=2):
            for cell in row:
                cell.alignment = Alignment(horizontal="center", vertical="center")

        # Обрабатываем данные текущего водителя
        driver_data = job_result_df[job_result_df[driver_name].notna()][['Day', 'Time', driver_name]]
        driver_data = driver_data.rename(columns={driver_name: 'Shift Details'})

        # Группируем действия по дням недели
        day_data = {day: [] for day in days_of_week}
        prev_action = None
        start_time = None
        prev_day = None
        prev_bus = None

        for _, row in driver_data.iterrows():
            current_time = row["Time"]
            shift_details = row["Shift Details"]
            action = shift_details[0] if len(shift_details) > 0 else None
            bus = shift_details[2].split(": ")[-1]  if len(shift_details) > 2 else None

            # Сопоставление дня недели
            day_name = days_of_week[row["Day"]]  # Исправлено: теперь день начинается с 0 для понедельника
            if action != prev_action or row["Day"] != prev_day:
                if prev_action is not None:
                    # Сохраняем данные для предыдущего действия
                    if "Закончил работу" in prev_action:
                        # Время конца = время начала для "Закончил работу"
                        day_data[days_of_week[prev_day]].append((prev_bus, start_time, start_time, prev_action))
                    else:
                        day_data[days_of_week[prev_day]].append((prev_bus, start_time, current_time, prev_action))
                start_time = current_time
                prev_action = action
                prev_bus = bus
                prev_day = row["Day"]


        # Сохраняем последнюю группу
        if prev_action is not None:
            day_name = days_of_week[row["Day"]]
            day_data[day_name].append((prev_bus, start_time, current_time, prev_action))

        # Записываем данные в лист
        row_idx = 3
        sheet.cell(row=row_idx, column=1, value=idx)  # Водитель
        sheet.cell(row=row_idx, column=2, value=shift_details[1])  # Вид смены

        max_len = max(len(day_data[day]) for day in days_of_week)

        for i in range(max_len):
            col_idx = 3
            for day in days_of_week:
                if i < len(day_data[day]):
                    bus, start, end, action = day_data[day][i]
                    # Проверка: Если автобус совпадает с предыдущим, оставляем пустую ячейку
                    if i == 0 or (i > 0 and bus != day_data[day][i - 1][0]):
                        sheet.cell(row=row_idx, column=col_idx, value=bus)
                    sheet.cell(row=row_idx, column=col_idx + 1, value=start)
                    sheet.cell(row=row_idx, column=col_idx + 2, value=end)
                    sheet.cell(row=row_idx, column=col_idx + 3, value=action)
                col_idx += 4
            row_idx += 1

        auto_adjust_column_width(sheet)
        add_borders(sheet)

    # Удаляем пустой стандартный лист
    if "Sheet" in workbook.sheetnames:
        workbook.remove(workbook["Sheet"])

    # Сохраняем файл
    workbook.save(output_file)

# Укажите путь к Excel файлу
output_file_path = "/content/drivers_week_schedule.xlsx"

# Вызов функции
excel_schedule(df, output_file_path)
